TRATAMENTO DE DADOS POR DATA

In [ ]:
#Tratamento de Data
import pandas as pd

df = pd.read_csv('movies.csv', 
                parse_dates=['release_date'], 
                dayfirst=False, 
                na_values=['', 'NA', 'NaN'], 
                dtype={'id': 'Int64'})


# Extrai ano, mês e dia da semana da data de lançamento
df['release_year']        = df['release_date'].dt.year
df['release_month']       = df['release_date'].dt.month
df['release_day_of_week'] = df['release_date'].dt.dayofweek  # Segunda=0, Domingo=6



# Verificação do Resultado
print("Visualização do DataFrame após o tratamento da data:")
print(df[['id', 'release_date', 'release_year', 'release_month', 'release_day_of_week']].head())

print(f"\nNúmero de datas inválidas ou faltantes: {df['release_date'].isnull().sum()}")

Visualização do DataFrame após o tratamento da data:
       id release_date  release_year  release_month  release_day_of_week
0  615656   2023-08-02        2023.0            8.0                  2.0
1  758323   2023-04-05        2023.0            4.0                  2.0
2  533535   2024-07-24        2024.0            7.0                  2.0
3  667538   2023-06-06        2023.0            6.0                  1.0
4  693134   2024-02-27        2024.0            2.0                  1.0

Número de datas inválidas ou faltantes: 51549
